# Introduction to Quantum Computing & Qiskit — Detailed Notes (Session 1)
**Course:** CS490/5590 — Quantum Computing Applications in Data Science, AI, & Deep Learning  
**Instructor:** Luke Miller  
**Level:** Graduate / Advanced Undergraduate  

> **Purpose of these notes.** A concise but thorough reference for Session 1. You’ll find crisp definitions, short proofs, worked examples, Qiskit snippets you can run, and mini-exercises with solutions sketched at the end of each section.

---

## Learning goals for this handout
By the end of this document you should be able to:
1. Explain what a qubit is and how it differs from a classical bit.  
2. Write single-qubit states in both Dirac (ket) and column-vector form.  
3. Compute measurement probabilities from amplitudes (Born rule).  
4. Describe superposition, unitaries, and why measurement “collapses” a state.  
5. Build and run a tiny Qiskit circuit (Hadamard + measurement) and interpret the results.  
6. Connect these primitives to why quantum is interesting for AI/DS (at a high level).

---

## 0. Course context & prerequisites (quick refresher)

**Focus.** We will use quantum foundations to motivate and prototype data-science/AI applications later in the course. Session 1 is about the minimal mathematical and programming machinery you need to read, write, and reason about small circuits.

**Prerequisites (what you actually need today):**
- **Linear algebra**: vectors, inner product, matrix–vector multiply, basic eigenvalue/eigenvector idea.
- **Python**: functions, imports, installing packages.
- **Calculus (preview use only)**: optimization shows up later in variational methods; not needed today.

> **Notation.** We will use Dirac (bra–ket) notation alongside matrices.  
> - $|\psi\rangle$ is a column vector (“ket”), $\langle \psi|$ is its conjugate transpose (“bra”).  
> - Inner product: $\langle \phi | \psi \rangle$.  
> - Normalization: $\langle \psi | \psi \rangle = 1$.

**Mini-exercise 0.1.** Show that if $U$ is unitary, $\|U|\psi\rangle\|=\||\psi\rangle\|$.  
*Sketch:* $\|U|\psi\rangle\|^2 = \langle\psi|U^\dagger U|\psi\rangle=\langle\psi|\psi\rangle$.

---

## 1. Why quantum computing?

Some tasks appear to require **exponential** resources classically (e.g., factoring, simulating quantum matter, some combinatorial optimization). Quantum devices manipulate information encoded in **quantum states**, which live in large complex vector spaces. This can yield **asymptotic speedups** for specific problems (e.g., quadratic in Grover’s search). For AI/DS, the promise centers on:
- Compactly representing high-dimensional structure (state spaces grow as $2^n$ with $n$ qubits).
- Using **interference** to emphasize/desuppress computational paths (a foundation for algorithmic advantage).
- Hybrid workflows (classical + quantum) that might reduce sample complexity or optimization steps in narrow settings.

> **Caution.** A speedup is **not** guaranteed in general. “Quantum ≠ faster” by default. We will be precise later about where and why advantages appear.

**Mini-exercise 1.1.** Distinguish “exponential Hilbert space” from “exponential speedup.” Why does the former not imply the latter in general?

---

## 2. From bits to qubits

### 2.1 Classical bits & gates (baseline)
- Bit: $b\in\{0,1\}$.  
- Gates: deterministic Boolean maps (AND/OR/NOT).  
- Matrix view of NOT (for comparison): $\begin{pmatrix}0&1\\1&0\end{pmatrix}$, acting on basis vectors $\begin{bmatrix}1\\0\end{bmatrix}$ (for 0) and $\begin{bmatrix}0\\1\end{bmatrix}$ (for 1).

### 2.2 Qubits: states as vectors
A **qubit** is a unit vector in a 2-dimensional complex Hilbert space:
$$
|\psi\rangle = \alpha|0\rangle + \beta|1\rangle,\qquad
|0\rangle=\begin{bmatrix}1\\0\end{bmatrix},\; |1\rangle=\begin{bmatrix}0\\1\end{bmatrix},\quad
|\alpha|^2+|\beta|^2=1.
$$
- $\alpha,\beta\in\mathbb{C}$ are **amplitudes**, not probabilities.
- Global phase $e^{i\theta}|\psi\rangle$ is physically indistinguishable from $|\psi\rangle$.

**Superposition example.**  
Equal superposition:
$$
|+\rangle \equiv \frac{|0\rangle+|1\rangle}{\sqrt{2}}
= \frac{1}{\sqrt{2}}\begin{bmatrix}1\\1\end{bmatrix}.
$$

### 2.3 Measurement (Born rule)
Measuring $\psi\rangle=\alpha|0\rangle+\beta|1\rangle$ in the $\{|0\rangle,|1\rangle\}$ basis returns  
- outcome “0” with probability $|\alpha|^2$; post-measurement state $|0\rangle$,  
- outcome “1” with probability$|\beta|^2$; post-measurement state $|1\rangle$.

Projection operator viewpoint:
$$
P_0 = |0\rangle\langle 0|,\quad \Pr(0)=\langle \psi|P_0|\psi\rangle=|\alpha|^2.
$$

> **Irreversibility & no-cloning.** Measurement is not unitary; it destroys amplitudes you don’t observe. The no-cloning theorem forbids copying an unknown quantum state exactly.

---

## 3. Unitary evolution and single-qubit gates

Quantum evolution (without measurement) is **unitary**: $U^\dagger U = I$. This preserves normalization and inner products.

Key single-qubit gates we’ll use immediately:
- **Pauli-X** (bit flip): $X=\begin{pmatrix}0&1\\1&0\end{pmatrix}$.  
  $X|0\rangle=|1\rangle,\; X|1\rangle=|0\rangle$.
- **Pauli-Z** (phase flip): $Z=\begin{pmatrix}1&0\\0&-1\end{pmatrix}$.  
  $Z|0\rangle=|0\rangle,\; Z|1\rangle=-|1\rangle$.
- **Hadamard** (creates superposition):
$$
H=\frac{1}{\sqrt{2}}\begin{pmatrix}1&1\\1&-1\end{pmatrix},\quad
H|0\rangle=|+\rangle,\; H|1\rangle=|-\rangle\equiv \frac{|0\rangle-|1\rangle}{\sqrt{2}}.
$$

> We will introduce multi-qubit gates (e.g., CNOT) and universality in Session 2. For now, focus on the one-qubit picture.

**Mini-exercise 3.1.** Verify $X^2=I$ and $X=X^\dagger$. Conclude $X$ is both unitary and Hermitian.

---

## 4. Superposition, interference, and “parallelism”

- **Superposition** means amplitudes over basis states.  
- **Interference** is what happens when amplitudes add (constructive) or cancel (destructive) after sequences of gates.  
- **Quantum parallelism** (informal): applying $U$ to $\sum_x \alpha_x|x\rangle$ updates **all components** at once. The trick is extracting something useful **with measurement**—which is where interference patterns are engineered.

**Tiny interference example (one qubit).**  
Start in $|0\rangle$. Apply $H$, then $Z$, then $H$ again:
$$
|0\rangle \xrightarrow{H} |+\rangle \xrightarrow{Z} \frac{|0\rangle - |1\rangle}{\sqrt{2}}=|-\rangle \xrightarrow{H} |1\rangle.
$$
Measurement will now return “1” with probability ~1. The middle $Z$ gate altered relative phase so that the final $H$ causes destructive interference on $|0\rangle$ and constructive on $|1\rangle$.

**Mini-exercise 4.1.** Show $H Z H = X$ and $H X H = Z$.

---

## 5. Minimal Qiskit: install, build, run, interpret



### 5.1 Installation
- Instructions for an online notebook.
```
!pip install qiskit qiskit-aer
```
If you see `ModuleNotFoundError: qiskit_aer`, ensure `qiskit-aer` installed successfully and your kernel restarted.

### 5.2 Your first circuit (Hadamard + measure)
```python
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

qc = QuantumCircuit(1, 1)   # 1 qubit, 1 classical bit
qc.h(0)                     # put qubit 0 into |+>
qc.measure(0, 0)            # measure into classical bit 0

sim = AerSimulator()
job = sim.run(transpile(qc, sim), shots=1000)
result = job.result()
counts = result.get_counts()
print(counts)               # ~ {'0': 500, '1': 500} up to sampling noise
```

**What you should see.** Approximately half the shots report `0` and half `1`. This reflects $|\alpha|^2=|\beta|^2=\tfrac12$.

### 5.3 Inspect the statevector (without measurement)
To **see** the amplitudes, evaluate the circuit **without** the final measurement:
```python
from qiskit.quantum_info import Statevector

qc2 = QuantumCircuit(1)
qc2.h(0)

psi = Statevector.from_label('0').evolve(qc2)
print(psi)          # Statevector([0.707+0.j, 0.707+0.j], dims=(2,))
print(psi.probabilities_dict())  # {'0': 0.5, '1': 0.5}
```
If you do measure, the simulator collapses the state and you only see classical outcomes.

### 5.4 A slightly richer interference demo (still 1 qubit)
```python
from qiskit.quantum_info import Statevector
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

qc3 = QuantumCircuit(1,1)
qc3.h(0)
qc3.z(0)
qc3.h(0)
qc3.measure(0,0)

sim = AerSimulator()
counts = sim.run(transpile(qc3, sim), shots=1000).result().get_counts()
print(counts)       # ~ {'1': 1000}
```
All shots should be `1` (up to numerical details), matching the hand calculation above.

> **Common gotchas (Qiskit):**  
> - Forgetting `transpile(qc, backend)` can slow or fail runs on real hardware (fine for simple sims).  
> - Measuring **after** you want to inspect amplitudes; measurement destroys the superposition.  
> - Bitstring order (endianness) matters for multi-qubit circuits (we’ll cover in Session 2).

---

## 6. Worked pencil-and-paper examples

### Example A — Normalize and measure
Let $ |\psi\rangle = \tfrac{1}{2}|0\rangle + \tfrac{\sqrt{3}}{2}|1\rangle$.
- Check normalization: $|\alpha|^2+|\beta|^2=\tfrac14+\tfrac34=1$.  
- Measurement probabilities: $\Pr(0)=\tfrac14,\; \Pr(1)=\tfrac34$.

### Example B — Track a state through a gate sequence
Start in $|0\rangle$. Apply $X$, then $H$.  
- After $X$: $|1\rangle$.  
- After $H$: $|-\rangle=(|0\rangle-|1\rangle)/\sqrt{2}$.  
- Measurement in $\{|0\rangle,|1\rangle\}$: $\Pr(0)=\Pr(1)=\tfrac12$ (same magnitudes),  
  but the **relative phase** (minus sign) will matter later when we compose more gates.

### Example C — Projector expectation
With $|\psi\rangle=\alpha|0\rangle+\beta|1\rangle$, the expected value of $Z$ is
$$
\langle Z \rangle = \langle \psi| Z |\psi \rangle
= |\alpha|^2 - |\beta|^2.
$$
For $|+\rangle$, $|\alpha|=|\beta|=\tfrac1{\sqrt{2}}\Rightarrow \langle Z\rangle=0$.

---

## 7. Why this matters for AI/DS (now, just the intuition)

- **Feature embedding via states.** A single qubit encodes two complex amplitudes; $n$ qubits encode a vector in $\mathbb{C}^{2^n}$. Carefully chosen encodings can place data into spaces where **simple** quantum operations correspond to **useful** nonlinear structure classically.  
- **Interference as computation.** Many circuits are engineered to produce interference patterns that are costly to emulate exactly, potentially giving specialized speedups.  
- **Hybrid is the norm.** For the foreseeable future, classical-quantum hybrids dominate (classical pre/post-processing with small quantum cores).

(We will **not** design quantum ML models today; this is context only.)

---

## 8. Practical setup checklist (for Homework 0)


1. **Smoke test** by running the 1-qubit Hadamard example above and confirming ~50/50 counts.  
2. (Optional) If you have time, re-run the interference demo and confirm ~100% of shots are `1`.  
3. **Submit** a short report (one screenshot of your histogram/counts + a 3–5 sentence explanation of why the distribution makes sense using the Born rule).

**Troubleshooting tips:**  
- If imports fail, restart your kernel/IDE after install.  
- If `qiskit-aer` wheels fail to build, upgrade `pip` and `setuptools` (`pip install -U pip setuptools wheel`) and reinstall.  
- If you’re on Apple Silicon or Windows with older Python, ensure a supported Python version.

---

## 9. Short exercises (with brief solutions)

**1. Amplitude vs probability.**  
State $|\psi\rangle=\alpha|0\rangle+\beta|1\rangle$ has $|\alpha|^2=0.2$. What is $\Pr(1)$?  
*Solution:* $\Pr(1)=|\beta|^2=1-0.2=0.8$.

**2. Normalization check.**  
Vector $\begin{bmatrix} \tfrac{1}{2} \\ \tfrac{\sqrt{3}}{2} \end{bmatrix}$. Is it normalized?  
*Solution:* Squares sum to $\tfrac14+\tfrac34=1$ → yes.

**3. Gate action.**  
Compute $H X |0\rangle$.  
*Solution:* $X|0\rangle=|1\rangle$. Then $H|1\rangle=|-\rangle=(|0\rangle-|1\rangle)/\sqrt{2}$.

**4. Probability from a circuit.**  
Suppose your Qiskit circuit does `qc.h(0); qc.z(0); qc.h(0); qc.measure(0,0)`. Predict the outcome distribution before running.  
*Solution:* Deterministic `1` (see interference demo).

**5. Write a circuit to prepare $|1\rangle$ starting from $|0\rangle$.**  
*Solution:* Apply `X` then measure; expect ~100% counts `1`.

**6. Reason about measurement.**  
You measured a qubit and got `0`. Can you “undo” the measurement to recover the prior superposition?  
*Solution:* No. Measurement is non-unitary; the prior amplitudes are gone.

---

## 10) FAQ / common pitfalls

- **“50/50 counts aren’t exact—did I do it wrong?”**  
  No—finite shots introduce sampling noise. Increase shots (e.g., 20k) to tighten proportions.

- **“Why are amplitudes complex?”**  
  Complex phases are essential for interference. Only magnitudes square to probabilities; phases influence *how* amplitudes combine under later gates.

- **“Why can’t we freely copy qubits for debugging?”**  
  No-cloning theorem: unknown quantum states can’t be copied without disturbing them.

- **“Why don’t we just measure early and often?”**  
  Measurement collapses the state, removing superposition resources that later gates would use.

- **“Is quantum always faster?”**  
  No. Known speedups are problem-dependent and often modest (e.g., quadratic). Many tasks likely see no advantage.

---

## 11) Summary (Session 1)

- A qubit is a normalized complex vector \(|\psi\rangle=\alpha|0\rangle+\beta|1\rangle\).  
- Unitary gates evolve states reversibly; measurement converts amplitudes to probabilities via the Born rule.  
- Superposition + interference enable algorithmic structure distinct from classical computing.  
- With Qiskit, you can build/run minimal circuits today: Hadamard → 50/50, simple interference → deterministic outcomes.  
- We’ll build on this to introduce multi-qubit systems and gates next session.

---

## 12) Preview of Session 2 (what to expect)

- Multi-qubit states via tensor products; basis ordering.  
- CNOT and entanglement (Bell states conceptually).  
- Gate matrices, universality, and simple circuit identities.  
- Measurement on multiple qubits; reading bitstrings.

*(We won’t do these now—this is just a heads up so today’s material has a “why next.”)*

---

## 13) Appendix: quick reference

**Basis kets & common states**
- $|0\rangle=\begin{bmatrix}1\\0\end{bmatrix}$
- $|1\rangle=\begin{bmatrix}0\\1\end{bmatrix}$.  
- $|\pm\rangle = (|0\rangle \pm |1\rangle)/\sqrt{2}$.

**Single-qubit gates**
- $X=\begin{pmatrix}0&1\\1&0\end{pmatrix}$,
- $Z=\begin{pmatrix}1&0\\0&-1\end{pmatrix}$
- $H=\tfrac{1}{\sqrt{2}}\begin{pmatrix}1&1\\1&-1\end{pmatrix}$.

**Born rule**
- If $|\psi\rangle=\alpha|0\rangle+\beta|1\rangle$, then $\Pr(0)=|\alpha|^2,\;\Pr(1)=|\beta|^2$.

**Qiskit mini-template**
```python
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

qc = QuantumCircuit(1,1)
# ... your gates ...
qc.measure(0,0)

backend = AerSimulator()
job = backend.run(transpile(qc, backend), shots=1000)
print(job.result().get_counts())
```

---

### Homework 0 (as assigned)
- Install Qiskit locally and run the **Hadamard + measurement** example.  
- Submit counts + a 3–5 sentence explanation using the Born rule.  
- Optional extension: run the interference example and explain why the result is deterministic.
